In [34]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F
import pdb
from PIL import Image
import os 
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torchvision
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from sklearn.preprocessing import StandardScaler
from modello import compute_output_size, calculate_metrics, CNN

In [35]:
dataset_no_correlazioni=pd.read_csv("dataset_no_correlazioni.csv")
colonne_riferimento = dataset_no_correlazioni.columns.tolist()

In [36]:
dataset_no_correlazioni.shape

(154293, 46)

In [37]:
nome_file="03-11/LDAP.csv"
with open(nome_file) as f:
    total_rows = sum(1 for _ in f) - 1  # Sottrai 1 per l'intestazione


n_sample = 10000
rows_to_sample = np.random.choice(total_rows, size=n_sample, replace=False)
skip_rows = sorted(set(range(1, total_rows + 1)) - set(rows_to_sample))
dataset = pd.read_csv(nome_file, skiprows=skip_rows)
dataset = dataset[colonne_riferimento]

dataset = dataset.replace([np.inf, -np.inf], np.nan)
dataset = dataset.dropna()

dataset[' Label'] = dataset[' Label'].map({'BENIGN': 0}).fillna(1).astype(int)

C:\Users\user\AppData\Local\Temp\ipykernel_31916\1692912899.py:9: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(nome_file, skiprows=skip_rows)


In [38]:
dataset.shape

(9737, 46)

In [39]:
labels = dataset[' Label']
features = dataset.drop(columns=[' Label'])

# Applico lo StandardScaler alle feature
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

# Crea un nuovo DataFrame con le feature normalizzate
dataset_normalized = pd.DataFrame(features_normalized, columns=features.columns)

# Aggiungi di nuovo la colonna 'Label' al dataset normalizzato
dataset_normalized[' Label'] = labels.reset_index(drop=True)

X = dataset_normalized.drop(columns=[' Label']).values
y = dataset_normalized[' Label'].values   # Converte le etichette in numeri

X_test_tensor = torch.FloatTensor(X)
y_test_tensor = torch.FloatTensor(y)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [40]:
path_coefficienti="model_weights.pth"
lunghezza_vettore = X.shape[1]
model = CNN(lunghezza_vettore=lunghezza_vettore).to(device)
model.load_state_dict(torch.load(path_coefficienti))

batch_size = 32
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                           shuffle=True)
loss_fn = nn.BCELoss()

C:\Users\user\AppData\Local\Temp\ipykernel_31916\2344883441.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path_coefficienti))


In [41]:
import torch
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Metti il modello in modalità di valutazione
model.eval()

# Variabili per tenere traccia delle predizioni e delle etichette reali
all_preds = []
all_labels = []
test_loss = 0.0

# Disattiva il calcolo dei gradienti
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device).float()
        
        # Ottieni le predizioni
        yhat = model(images).squeeze(1)
        
        # Calcola il loss
        loss = loss_fn(yhat, labels)
        test_loss += loss.item()
        
        # Applica una soglia di 0.5 per classificare come 0 o 1 (adatto per output di tipo sigmoid)
        preds = (yhat > 0.5).float()
        
        # Memorizza le predizioni e le etichette vere
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calcola il loss medio sul test set
avg_test_loss = test_loss / len(test_loader)

# Calcola altre metriche
accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)

# Stampa le metriche
print(f"Test Loss: {avg_test_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")

# Calcola e visualizza la matrice di confusione
conf_matrix = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Benign", "Malware"], yticklabels=["Benign", "Malware"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title('Confusion Matrix')

base_name = os.path.basename(nome_file)  # Ottiene 'LDAP.csv'
name_without_ext = os.path.splitext(base_name)[0]  # Ottiene 'LDAP'
output_file_name = f'confusion_matrix_{name_without_ext}.png'
plt.savefig(output_file_name)
plt.show()


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x432 and 480x1)